In [ ]:
%matplotlib inline
from ROOT import TFile, TTree

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import re

from root_numpy import root2array


In [ ]:
filedir = '/Users/davidkaleko/larlite/UserDev/KalekoAna/CCInclusive/mac/polished_output/'
!ls $filedir

In [ ]:
mcnu = 'XiaoEventAna_out_mcc7_BNBcosmic_NOFILTER.root'
mcbkg = 'XiaoEventAna_out_INTIMECOSMIC.root'
datanu = 'XiaoEventAna_out_BNBFilter.root'
databkg = 'XiaoEventAna_out_BNBEXT.root'

files = { 'mcnu' : filedir + mcnu, 
          'mcbkg' : filedir + mcbkg,
         'datanu' : filedir + datanu, 
         'databkg' : filedir + databkg }

mycuts = ['nu_E_estimate > 2.5','second_longest_trk_len > 16.',
          'longest_tracks_dotprod_trkendpoints > -0.5']
#mcbkgcut = 'correct_ID == 0'
#bkgnorm = 1.2

In [ ]:
df_dict, n_evts_analyzed = {}, {}

for sample, filepath in files.iteritems():
    f = open(filepath[:-5]+'_log.txt')
    for line in f:
        if 'Processed' not in line: continue
        m = re.search('(\d+)\/(\d+)', line)
        assert m
        n_evts_analyzed[sample] = int(m.group(1))
    f.close()

    df_dict[sample] = pd.DataFrame( root2array ( filepath, 'tree' ) )

In [ ]:
pot_per_sample = { 'mcnu' : (1.2105e15) * n_evts_analyzed['mcnu'],
                   'datanu' : 4.88e19 }
pot_per_sample['databkg'] = pot_per_sample['datanu'] * 0.844 * \
                  float(n_evts_analyzed['datanu'])/n_evts_analyzed['databkg']
pot_per_sample['mcbkg'] = pot_per_sample['databkg'] * (1./0.336) * \
                  float(n_evts_analyzed['mcbkg'])/n_evts_analyzed['databkg'] 

In [ ]:
print n_evts_analyzed
print pot_per_sample

In [ ]:
print df_dict.keys()

In [ ]:
print df_dict['datanu'].columns.values
vars_to_try = ['longest_trk_len','nu_E_estimate','longest_trk_theta',\
               'longest_trk_MCS_mom', 'n_associated_tracks']
#df_dict['mcnu'].hist('fndecay')

In [ ]:
def plotVariableComparison(myvar, mybins, myquery, mytitle, myshapeonly = False, myylims = None,\
                           myxlabel = 'test', myylabel = 'test'):

    plt.figure(figsize=(10,6))
    poop = plt.grid(True)
    plt.title(mytitle,fontsize=16)
    #plt.ylabel('Events: 5e19 POT Normalized',fontsize=16)
    
    mydict = df_dict['mcnu']
    if myquery: mydict = df_dict['mcnu'].query(myquery)
    #myvals = mydict[myvar].values
    myweight = (5.e19) / pot_per_sample['mcnu']
    
    myvals_pion = mydict.query('fndecay >= 10 and correct_ID == 1')[myvar].values
    myvals_kaon = mydict.query('fndecay < 10 and correct_ID == 1')[myvar].values
    myvals_cosm = mydict.query('correct_ID == 0')[myvar].values
    #integral = float(len(myvals))*myweight
    nphist = np.histogram(myvals_cosm,bins=mybins,
                          weights=[myweight]*len(myvals_cosm),
                          normed=myshapeonly)
    integral_cosm = np.sum(nphist[0])  
    nphist = np.histogram(myvals_pion,bins=mybins,
                          weights=[myweight]*len(myvals_pion),
                          normed=myshapeonly)
    integral_pion = np.sum(nphist[0])   
    nphist = np.histogram(myvals_kaon,bins=mybins,
                          weights=[myweight]*len(myvals_kaon),
                          normed=myshapeonly)
    integral_kaon = np.sum(nphist[0])

    #mydict_intime = df_dict['mcbkg']
    #if myquery: mydict_intime = df_dict['mcbkg'].query(myquery)
    #myweight_intime = (5.e19) / pot_per_sample['mcbkg']
    #myvals_intime = mydict_intime[myvar].values
    #nphist = np.histogram(myvals_intime,bins=mybins,
    #                      weights=[myweight_intime]*len(myvals_intime),
    #                      normed=myshapeonly)
    #integral_intime = np.sum(nphist[0])
    
    poop = plt.hist([myvals_cosm,myvals_pion,myvals_kaon],bins=mybins,
                    label=[#'MC: Cosmic Bkg. from In Time Cosmic Entries = %0.2f' % integral_intime,\
                           'MC: Cosmic Bkg. from BNB+Cosmic Entries = %0.2f' % integral_cosm, \
                           'MC: Numu from Pion Bkg. Entries = %0.2f' % integral_pion, \
                           'MC: Kaon Signal. Entries = %0.2f\n    Total MC = (%0.2f)' \
                           % (integral_kaon,integral_cosm+integral_pion+integral_kaon)],
                    alpha=0.5,
                    weights=[#[myweight_intime]*len(myvals_intime),
                             [myweight]*len(myvals_cosm),
                             [myweight]*len(myvals_pion),
                             [myweight]*len(myvals_kaon)],
                             normed=myshapeonly,
                   color=['r','b','g'],
                    stacked=True,
                   rwidth=1.)

    mybnbdict = df_dict['datanu']    
    if myquery: mybnbdict = df_dict['datanu'].query(myquery)
    mybnbvals = mybnbdict[myvar].values
    mybnbweight = (5.e19) / pot_per_sample['datanu']
    
    blah = plt.hist(mybnbvals,bins=mybins,color='g',
                    alpha=0,weights=[mybnbweight]*len(mybnbvals),normed=myshapeonly)
    
    ybnbvals = blah[0]
    xbnbvals = [blah[1][i]+(blah[1][i+1]-blah[1][i])/2. for i in xrange(len(blah[1][:-1]))]
    bnbintegral = np.sum(blah[0])
    #awefia = plt.plot(xbnbvals,ybnbvals,'bo',
    #                  label='BNB DATA: Entries = %0.2f' % bnbintegral
    #                 )

    myextvals = df_dict['databkg'][myvar].values
    if myquery: myextvals = df_dict['databkg'].query(myquery)[myvar].values
    myextweight = (5.e19) / pot_per_sample['databkg']
    extintegral = 0.
    if len(myextvals):
        blah = plt.hist(myextvals,bins=mybins,color='g',
                        alpha=0,weights=[myextweight]*len(myextvals),normed=myshapeonly)
    
        yextvals = blah[0]
        xextvals = [blah[1][i]+(blah[1][i+1]-blah[1][i])/2. for i in xrange(len(blah[1][:-1]))]
        yerrs = np.sqrt(np.array(yextvals)*myextweight)
        #extintegral = float(len(myextvals))*myextweight
        extintegral = np.sum(blah[0])
        #awefia = plt.errorbar(xextvals,yextvals,fmt='ro', yerr=yerrs,
        #                  label='BNB EXT DATA: Entries = %0.2f' % extintegral
        #                 )

        diffintegral = bnbintegral-extintegral
        yerrs = np.sqrt(np.array(ybnbvals)*mybnbweight + np.array(yextvals)*myextweight)
        #if yerrs is zero, set equal to 1 event
        yerrs = [x if x else 1 for x in yerrs]
        awefia = plt.errorbar(xextvals,ybnbvals-yextvals,fmt='mo',yerr = yerrs,
                          label='BNB DATA - BNB EXT DATA (%0.2f)' % diffintegral
                         )
    else:
        diffintegral = bnbintegral
        yerrs = np.sqrt(np.array(ybnbvals)*mybnbweight)
        #if yerrs is zero, set equal to 1 event
        yerrs = [x if x else 1 for x in yerrs]
        awefia = plt.errorbar(xbnbvals,ybnbvals,fmt='mo',yerr = yerrs,
                          label='BNB DATA - BNB EXT DATA (%0.2f)' % diffintegral
                         )
    plt.ylim((0, plt.ylim()[1]))
    if myylims is not None:
        plt.ylim(myylims)
    leg = plt.legend()
    plt.xlabel(myxlabel,fontsize=16)
    plt.ylabel(myylabel,fontsize=16)
    leg.get_frame().set_alpha(0.5)

In [ ]:
def loopCutsPlots(myvar, mybins, mytitlebase, myshapeonly, myxlabel, myylabel):

    myquery = ''
    mytitle = mytitlebase + ': NO CUTS'
    plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,\
                          myxlabel=myxlabel,myylabel=myylabel)
    
    for myquery in mycuts:
        mytitle = mytitlebase + ': %s' % myquery
        plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,\
                              myxlabel=myxlabel,myylabel=myylabel)

    myquery = str.join(' and ',mycuts)
    mytitle = mytitlebase + ': ALL CUTS'
    plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,\
                           myxlabel=myxlabel,myylabel=myylabel)

In [ ]:
myvar = 'longest_trk_len'
mybins = np.linspace(0,1000,20)
mytitlebase = 'Longest Track Length'
myshapeonly = False
myxlabel = 'Length of Longest Associated Track [cm]'
myylabel = 'Events (5e19 POT Normalized)'
loopCutsPlots(myvar,mybins,mytitlebase,myshapeonly,myxlabel=myxlabel,myylabel=myylabel)

In [ ]:
myvar = 'nu_E_estimate'
mybins = np.linspace(0,5,50)
mytitlebase = 'Estimated Nu Energy'
myshapeonly = False
myxlabel = 'Cruedly Reconstructed Neutrino Energy [GeV]'
myylabel = 'Events (5e19 POT Normalized)'
loopCutsPlots(myvar,mybins,mytitlebase,myshapeonly,myxlabel,myylabel)

In [ ]:
myvar = 'longest_trk_theta'
mybins = np.linspace(0,3.1415/2.,20)
mytitlebase = 'Longest Track Theta'
myshapeonly = False
myxlabel = 'Theta Angle of Longest Track (w.r.t. Beam) [Radians]'
myylabel = 'Events (5e19 POT Normalized)'
loopCutsPlots(myvar,mybins,mytitlebase,myshapeonly,myxlabel,myylabel)

In [ ]:
myvar = 'longest_trk_MCS_mom'
mybins = np.linspace(0,3000,50)
mytitlebase = 'Longest Track MCS Momentum'
myshapeonly = False
myxlabel = 'Longest Track MCS Momentum [MeV]'
myylabel = 'Events (5e19 POT Normalized)'
loopCutsPlots(myvar,mybins,mytitlebase,myshapeonly,myxlabel,myylabel)

In [ ]:
myvar = 'n_associated_tracks'
mybins = np.linspace(-0.5,7.5,9)
mytitlebase = 'Track Multiplicity'
myshapeonly = False
myxlabel = 'Track Multiplicity'
myylabel = 'Events (5e19 POT Normalized)'
loopCutsPlots(myvar,mybins,mytitlebase,myshapeonly,myxlabel,myylabel)

In [ ]:
def computeCutImpact(myquery1 = None, myquery2 = None):
    """
    Query 1 is first query (for example, NO cuts)
    Query 2 is the second query, whose impact you are quantifying
    """
    print
    print "COMPUTING CUT IMPACT!"
    print "Base Query = %s" % ('No Cuts' if not myquery1 else myquery1)
    print "Placing Cut = %s" % myquery2
    print
    mydict = df_dict['mcnu']
    if myquery1 is not None: mydict = df_dict['mcnu'].query(myquery1)
    myweight = (5.e19) / pot_per_sample['mcnu']  
    myinum_pion = len(mydict.query('fndecay >= 10 and correct_ID == 1'))*float(myweight)
    myinum_kaon = len(mydict.query('fndecay < 10 and correct_ID == 1'))*float(myweight)
    myinum_cosm = len(mydict.query('correct_ID == 0'))*float(myweight)
    
    myfnum_pion = len(mydict.query('fndecay >= 10 and correct_ID == 1 and %s'%myquery2))*float(myweight)
    myfnum_kaon = len(mydict.query('fndecay < 10 and correct_ID == 1 and %s'%myquery2))*float(myweight)
    myfnum_cosm = len(mydict.query('correct_ID == 0 and %s'%myquery2))*float(myweight)

    mydict = df_dict['datanu']
    if myquery1 is not None: mydict = df_dict['datanu'].query(myquery1)
    myweight = (5.e19) / pot_per_sample['datanu']  
    myinum_bnb = len(mydict)
    myfnum_bnb = len(mydict.query(myquery2))
    
    mydict = df_dict['databkg']
    if myquery1 is not None: mydict = df_dict['databkg'].query(myquery1)
    myweight = (5.e19) / pot_per_sample['databkg']  
    myinum_ext = len(mydict)
    myfnum_ext = len(mydict.query(myquery2)) 
    
    print "\tN_Pion\tN_Cosm\tN_Kaon\tN_BNB\tN_EXT\tN_MCTot\tN_Sig-Bkg"
    print "N_EVTS\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f" % \
    (myinum_pion,myinum_cosm,myinum_kaon,myinum_bnb,myinum_ext,\
     myinum_pion+myinum_cosm+myinum_kaon,myinum_bnb-myinum_ext)
    print "AFTER\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f\t%0.1f" % \
    (myfnum_pion,myfnum_cosm,myfnum_kaon,myfnum_bnb,myfnum_ext,\
     myfnum_pion+myfnum_cosm+myfnum_kaon,myfnum_bnb-myfnum_ext)
    print "Remain\t%0.1f%%\t%0.1f%%\t%0.1f%%\t%0.1f%%\t%0.1f%%\t%0.1f%%\t%0.1f%%" % \
    (100.*myfnum_pion/myinum_pion,100.*myfnum_cosm/myinum_cosm,\
     100.*myfnum_kaon/myinum_kaon,100.*myfnum_bnb/myinum_bnb,\
     100.*myfnum_ext/myinum_ext,\
    100.*(myfnum_pion+myfnum_cosm+myfnum_kaon)/(myinum_pion+myinum_cosm+myinum_kaon),\
     100.*(myfnum_bnb-myfnum_ext)/(myinum_bnb-myinum_ext))
    print
    
    ipdiff = 100.*(((myinum_pion+myinum_cosm+myinum_kaon)-(myinum_bnb-myinum_ext))/(myinum_bnb-myinum_ext))
    fpdiff = 100.*(((myfnum_pion+myfnum_cosm+myfnum_kaon)-(myfnum_bnb-myfnum_ext))/(myfnum_bnb-myfnum_ext)) 
    print "Before cuts, percent difference b/t data and MC is %0.1f%%" % ipdiff
    print "After cuts, percent difference b/t data and MC is %0.1f%%" % fpdiff
       
    

In [ ]:
for cut in mycuts:
    computeCutImpact(myquery1=None,myquery2=cut)

In [ ]:
print list(df_dict['datanu'].columns.values)
print mycuts

In [ ]:
myvar = 'longest_tracks_dotprod_trkendpoints'
mybins = np.linspace(-1,1,20)
mytitle = myvar
myshapeonly = False
myquery = ''
myylims = (0,350)
plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,myylims)
plt.axvline(x=-0.5, ymin=0, ymax=350, linewidth=4,color='red')

myvar = 'nu_E_estimate'
mybins = np.linspace(0,5,20)
mytitle = 'Estimated Nu Energy'
myshapeonly = False
myxlabel = 'Cruedly Reconstructed Neutrino Energy [GeV]'
myylabel = 'Events (5e19 POT Normalized)'
myquery = ''
myylims = (0,1000)
plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,myylims)
plt.axvline(x=2.5, ymin=0, ymax=1000, linewidth=4,color='red')

myvar = 'nu_E_estimate'
mybins = np.linspace(2.5,5,20)
mytitle = 'Estimated Nu Energy'
myshapeonly = False
myxlabel = 'Cruedly Reconstructed Neutrino Energy [GeV]'
myylabel = 'Events (5e19 POT Normalized)'
myquery = ''
myylims = (0,20)
plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,myylims)
plt.axvline(x=2.5, ymin=0, ymax=1000, linewidth=4,color='red')

myvar = 'second_longest_trk_len'
mybins = np.linspace(0,150,40)
mytitle = 'Second Longest Track Length'
myshapeonly = False
myquery = ''
myylims = (0,600)
myxlabel = 'Second Longest Track Length [cm]'
myylabel = 'Events (5e19 POT Normalized)'
plotVariableComparison(myvar,mybins,myquery,mytitle,myshapeonly,myylims)
line = plt.axvline(x=16, ymin=0, ymax=600, linewidth=4,color='red')

In [ ]:
def plotVariableComparisonBkg(myvar, mybins, myquery, mytitle, myshapeonly = False, myylims = None,\
                           myxlabel = 'test', myylabel = 'test'):

    plt.figure(figsize=(10,6))
    poop = plt.grid(True)
    plt.title(mytitle,fontsize=16)
  
    mydict_intime = df_dict['mcbkg']
    if myquery: mydict_intime = df_dict['mcbkg'].query(myquery)
    myweight_intime = (5.e19) / pot_per_sample['mcbkg']
    myvals_intime = mydict_intime[myvar].values
    nphist = np.histogram(myvals_intime,bins=mybins,
                          weights=[myweight_intime]*len(myvals_intime),
                          normed=myshapeonly)
    integral_intime = np.sum(nphist[0])
    
    poop = plt.hist(myvals_intime,bins=mybins,
                    label='MC: Cosmic Bkg. from In Time Cosmic Entries = %0.2f' % integral_intime,
                    alpha=0.5,
                    weights=[myweight_intime]*len(myvals_intime),
                    color='cyan',
                    stacked=False,
                    rwidth=1.)

    myextvals = df_dict['databkg'][myvar].values
    if myquery: myextvals = df_dict['databkg'].query(myquery)[myvar].values
    myextweight = (5.e19) / pot_per_sample['databkg']
    extintegral = 0.
    if len(myextvals):
        blah = plt.hist(myextvals,bins=mybins,color='g',
                        alpha=0,weights=[myextweight]*len(myextvals),normed=myshapeonly)
    
        yextvals = blah[0]
        xextvals = [blah[1][i]+(blah[1][i+1]-blah[1][i])/2. for i in xrange(len(blah[1][:-1]))]
        yerrs = np.sqrt(np.array(yextvals)*myextweight)
        extintegral = np.sum(blah[0])
        awefia = plt.errorbar(xextvals,yextvals,fmt='ro', yerr=yerrs,
                          label='BNB EXT DATA: Entries = %0.2f' % extintegral
                         )

    plt.ylim((0, plt.ylim()[1]))
    if myylims is not None:
        plt.ylim(myylims)
    leg = plt.legend()
    plt.xlabel(myxlabel,fontsize=16)
    plt.ylabel(myylabel,fontsize=16)
    dummy = leg.get_frame().set_alpha(0.5)

In [ ]:
myvar = 'longest_tracks_dotprod_trkendpoints'
mybins = np.linspace(-1,1,20)
mytitle = 'Longest Two Tracks: Direction Dot-Product'
myshapeonly = False
myquery = ''
myxlabel = 'Dot Product of Longest Two Track Directions'
myylabel = 'Events (5e19 POT Normalized)'
myylims = (0,80)
plotVariableComparisonBkg(myvar,mybins,myquery,mytitle,myshapeonly,myylims,myxlabel,myylabel)
plt.axvline(x=-0.5, ymin=0, ymax=350, linewidth=4,color='red')

myvar = 'nu_E_estimate'
mybins = np.linspace(0,5,20)
mytitle = 'Estimated Nu Energy'
myshapeonly = False
myxlabel = 'Cruedly Reconstructed Neutrino Energy [GeV]'
myylabel = 'Events (5e19 POT Normalized)'
myquery = ''
myylims = (0,300)
plotVariableComparisonBkg(myvar,mybins,myquery,mytitle,myshapeonly,myylims,myxlabel,myylabel)
plt.axvline(x=2.5, ymin=0, ymax=1000, linewidth=4,color='red')

myvar = 'nu_E_estimate'
mybins = np.linspace(2.5,5,20)
mytitle = 'Estimated Nu Energy'
myshapeonly = False
myxlabel = 'Cruedly Reconstructed Neutrino Energy [GeV]'
myylabel = 'Events (5e19 POT Normalized)'
myquery = ''
myylims = (0,10)
plotVariableComparisonBkg(myvar,mybins,myquery,mytitle,myshapeonly,myylims,myxlabel,myylabel)
plt.axvline(x=2.5, ymin=0, ymax=1000, linewidth=4,color='red')

myvar = 'second_longest_trk_len'
mybins = np.linspace(0,150,40)
mytitle = 'Second Longest Track Length'
myshapeonly = False
myquery = ''
myylims = (0,250)
myxlabel = 'Second Longest Track Length [cm]'
myylabel = 'Events (5e19 POT Normalized)'
plotVariableComparisonBkg(myvar,mybins,myquery,mytitle,myshapeonly,myylims,myxlabel,myylabel)
line = plt.axvline(x=16, ymin=0, ymax=600, linewidth=4,color='red')